In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [3]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.2 MB/s eta 0:00:00


In [4]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=f5d1e0a164fd2eab662328faec7d3891928a277348f3ef17b72a79989b4cd55e
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [5]:
pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [6]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer, AutoModelForSequenceClassification
from datasets import Dataset
from evaluate import load
from torch.optim import AdamW
from tqdm.auto import tqdm
import time
import locale

In [13]:
# # Fix encoding issues
# def getpreferredencoding(do_setlocale=True):
#     return "UTF-8"
# locale.getpreferredencoding = getpreferredencoding

# Check device
device = "cuda" if torch.cuda.is_available() else "cpu"

In [14]:
device

'cpu'

In [8]:
# Load dataset
df = pd.read_csv('/content/drive/MyDrive/NLP_project_1/tarot_readings_final.csv')

# Shuffle the dataset for randomness
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Display basic information
print(df.columns)
print(df.head())


Index(['zodiac', 'question', 'card1', 'card2', 'card3', 'reading'], dtype='object')
     zodiac                                           question  \
0       Leo              How can I overcome burnout in school?   
1  Aquarius     How can I create new traditions for my family?   
2     Libra    What role does money play in my life's purpose?   
3     Aries               What is my soul guiding me to study?   
4    Cancer  How can I connect my education to my life purp...   

                card1             card2          card3  \
0     Three of Swords       The Empress   King of Cups   
1  The High Priestess         The Tower      The Devil   
2       Seven of Cups      Page of Cups  King of Wands   
3   Four of Pentacles   Knight of Wands        Justice   
4           The World  Ten of Pentacles    Ace of Cups   

                                             reading  
0  Reading:  The Three of Swords suggests you're ...  
1  Reading:  Aquarius, your intuition (High Pries...  
2  Li

In [9]:
# Load Twitter-based sentiment analysis model
sentiment_model_name = "cardiffnlp/twitter-roberta-base-sentiment"
sentiment_tokenizer = AutoTokenizer.from_pretrained(sentiment_model_name)
sentiment_model = AutoModelForSequenceClassification.from_pretrained(sentiment_model_name).to(device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [10]:
# Map label ids
sentiment_labels = {0: "negative", 1: "neutral", 2: "positive"}

# Sentiment prediction function
def predict_sentiment(text):
    inputs = sentiment_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    outputs = sentiment_model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    pred = torch.argmax(probs, dim=-1).item()
    return sentiment_labels[pred]



In [11]:
# Apply to user question
df['question_sentiment'] = df['question'].apply(lambda x: predict_sentiment(str(x)))

In [14]:
# Check unique value counts for question sentiment
print(df['question_sentiment'].value_counts())


question_sentiment
neutral     9757
negative    1736
positive     497
Name: count, dtype: int64


In [16]:
df.to_csv('/content/drive/MyDrive/NLP_project_1/tarot_readings_with_sentiment.csv', index=False)

In [20]:
df = pd.read_csv('/content/drive/MyDrive/NLP_project_1/tarot_readings_with_sentiment.csv')


In [16]:
# # Sample 1000 random rows
# df = df.sample(n=1000, random_state=42).reset_index(drop=True)

# Display
print(df.head())

     zodiac                                           question  \
0       Leo              How can I overcome burnout in school?   
1  Aquarius     How can I create new traditions for my family?   
2     Libra    What role does money play in my life's purpose?   
3     Aries               What is my soul guiding me to study?   
4    Cancer  How can I connect my education to my life purp...   

                card1             card2          card3  \
0     Three of Swords       The Empress   King of Cups   
1  The High Priestess         The Tower      The Devil   
2       Seven of Cups      Page of Cups  King of Wands   
3   Four of Pentacles   Knight of Wands        Justice   
4           The World  Ten of Pentacles    Ace of Cups   

                                             reading question_sentiment  
0  Reading:  The Three of Swords suggests you're ...           negative  
1  Reading:  Aquarius, your intuition (High Pries...            neutral  
2  Libra, your imagination offer

In [21]:
def create_prompt(row):
    prompt = (
        f"You are a Tarot Master.\n"
        f"User Zodiac Sign: {row['zodiac']}\n"
        f"User Sentiment: {row['question_sentiment']}\n"
        f"User's Question: {row['question']}\n"
        f"Drawn Cards: {row['card1']}, {row['card2']}, {row['card3']}\n"
        f"Based on these, provide a detailed, sensitive, and inspiring Tarot Reading for each card and then summarized scentence at the end :"
    )
    return prompt


In [22]:

df['input_text'] = df.apply(create_prompt, axis=1)
df['target_text'] = df['reading']# Ground truth

# Keep only input and target
df = df[['input_text', 'target_text']]

# Convert to Huggingface Dataset
dataset = Dataset.from_pandas(df)

In [23]:
seed =42
# Split into 80% train, 10% validation, 10% test directly
train_val_test = dataset.train_test_split(test_size=0.2, seed=seed)
train_set = train_val_test['train']
temp_set = train_val_test['test']

# Now split temp_set into validation and test
val_test_split = temp_set.train_test_split(test_size=0.5, seed=seed)
validation_set = val_test_split['train']
test_set = val_test_split['test']

# Assign to dataset dict
dataset = {
    "train": train_set,
    "validation": validation_set,
    "test": test_set
}


In [11]:
# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")

# Tokenization function
def process_data_to_model_inputs(batch):
    inputs = tokenizer(batch["input_text"], padding="max_length", truncation=True, max_length=512)
    outputs = tokenizer(batch["target_text"], padding="max_length", truncation=True, max_length=512)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["labels"] = outputs.input_ids
    return batch

# Apply tokenization
dataset['train'] = dataset['train'].map(process_data_to_model_inputs, batched=True)
dataset['validation'] = dataset['validation'].map(process_data_to_model_inputs, batched=True)
dataset['test'] = dataset['test'].map(process_data_to_model_inputs, batched=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/9592 [00:00<?, ? examples/s]

Map:   0%|          | 0/1199 [00:00<?, ? examples/s]

Map:   0%|          | 0/1199 [00:00<?, ? examples/s]

In [17]:
def collate_fn(batch):
    batch = {key: [d[key] for d in batch] for key in batch[0]}
    batch["input_ids"] = torch.tensor(batch["input_ids"]).to(device)
    batch["attention_mask"] = torch.tensor(batch["attention_mask"]).to(device)
    batch["labels"] = torch.tensor(batch["labels"]).to(device)
    return batch

train_loader = DataLoader(dataset['train'], batch_size=4, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(dataset['validation'], batch_size=4, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(dataset['test'], batch_size=4, shuffle=False, collate_fn=collate_fn)


In [18]:
# Load model
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
model.to(device)

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [19]:
def validate(model, val_loader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            outputs = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                labels=batch["labels"]
            )
            loss = outputs.loss
            total_loss += loss.item()
    return total_loss / len(val_loader)



In [20]:
num_epoch = 5
prev_val_loss = float('inf')

for epoch in range(num_epoch):
    model.train()
    total_loss = 0
    start_time = time.time()

    for batch in train_loader:
        optimizer.zero_grad()
        outputs = model(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            labels=batch['labels']
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        torch.cuda.empty_cache()

    end_time = time.time()
    epoch_duration = end_time - start_time
    formatted_time = time.strftime("%H:%M:%S", time.gmtime(epoch_duration))

    val_loss = validate(model, val_loader)
    print(f"Epoch {epoch+1}, Train Loss: {total_loss/len(train_loader):.4f} | Val Loss: {val_loss:.4f} | Duration: {formatted_time}")

    if val_loss < prev_val_loss:
        print(f"{val_loss:.4f} < {prev_val_loss:.4f} - Saving model...")
        model.save_pretrained("/content/drive/MyDrive/NLP_project_1/tarot_model")
        tokenizer.save_pretrained("/content/drive/MyDrive/NLP_project_1/tarot_model")
        prev_val_loss = val_loss


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch 1, Train Loss: 0.5883 | Val Loss: 0.1926 | Duration: 00:12:13
0.1926 < inf - Saving model...
Epoch 2, Train Loss: 0.2004 | Val Loss: 0.1705 | Duration: 00:12:13
0.1705 < 0.1926 - Saving model...
Epoch 3, Train Loss: 0.1786 | Val Loss: 0.1552 | Duration: 00:12:13
0.1552 < 0.1705 - Saving model...
Epoch 4, Train Loss: 0.1618 | Val Loss: 0.1441 | Duration: 00:12:12
0.1441 < 0.1552 - Saving model...
Epoch 5, Train Loss: 0.1485 | Val Loss: 0.1364 | Duration: 00:12:12
0.1364 < 0.1441 - Saving model...


In [24]:
def predict_tarot_reading(zodiac, sentiment, question, card1, card2, card3):
    prompt = (
        f"You are a Tarot Master.\n"
        f"User Zodiac Sign: {zodiac}\n"
        f"User Sentiment: {sentiment}\n"
        f"User's Question: {question}\n"
        f"Drawn Cards: {card1}, {card2}, {card3}\n"
        f"Based on these, provide a detailed, sensitive, and inspiring Tarot Reading for each card and then summarized scentence at the end :"
    )

    load_model.eval()
    with torch.no_grad():
        inputs = load_tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)
        outputs = load_model.generate(
            input_ids=inputs["input_ids"].to(device),
            attention_mask=inputs["attention_mask"].to(device),
            max_length=1028,
            temperature=0.7,
            do_sample=True,
            top_k=20
        )
        reading = load_tokenizer.decode(outputs[0], skip_special_tokens=True)

    return reading


In [27]:
# Example
print(predict_tarot_reading("Cancer", "positive", "How will be my carrer path in next year?", "The Moon", "The Devil", "Ace of Swords"))


Reading: Cancer, your career path involves hidden anxieties (The Moon) and unhealthy attachments (The Devil). A new, clear perspective will open your mind (Ace of Swords). Focus on clear communication and emotional intelligence.


In [24]:
from evaluate import load

rouge = load('rouge')
bertscore = load('bertscore')

load_model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/NLP_project_1/tarot_model")
load_tokenizer = T5Tokenizer.from_pretrained("/content/drive/MyDrive/NLP_project_1/tarot_model")
load_model.to(device)
load_model = load_model.eval()



In [25]:
# Generation function
def generate_predictions(batch):
    inputs = load_tokenizer(batch["input_text"], return_tensors="pt", padding="max_length", truncation=True, max_length=512)
    outputs = load_model.generate(
        input_ids=inputs["input_ids"].to(device),
        attention_mask=inputs["attention_mask"].to(device),
        max_length=512,
        temperature=0.7,
        do_sample=True,
        top_k=20
    )
    batch["generated_text"] = load_tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return batch

# Apply generation
results = dataset['test'].map(generate_predictions, batched=True, batch_size=4)

Map:   0%|          | 0/1199 [00:00<?, ? examples/s]

In [26]:
# ROUGE Score
rouge_score = rouge.compute(predictions=results["generated_text"], references=results["target_text"])
print("ROUGE Score:", rouge_score)

# BERTScore
bertscore_result = bertscore.compute(predictions=results["generated_text"], references=results["target_text"], lang="en")
print("BERTScore F1:", bertscore_result['f1'])


ROUGE Score: {'rouge1': np.float64(0.4600167526822335), 'rouge2': np.float64(0.24000331346221276), 'rougeL': np.float64(0.3719526787691789), 'rougeLsum': np.float64(0.371865664531577)}


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore F1: [0.9005871415138245, 0.9214460253715515, 0.8797044157981873, 0.9266871213912964, 0.8968577980995178, 0.900276243686676, 0.886903703212738, 0.879358172416687, 0.8803215026855469, 0.9129812717437744, 0.9145448207855225, 0.909030556678772, 0.9032915830612183, 0.9071760773658752, 0.8752471208572388, 0.9010807275772095, 0.9198383092880249, 0.8850657343864441, 0.9107358455657959, 0.8771969676017761, 0.8638975024223328, 0.887825071811676, 0.9081975817680359, 0.9042380452156067, 0.903207004070282, 0.9070888757705688, 0.887112021446228, 0.8780189156532288, 0.8907067775726318, 0.9296488165855408, 0.8943953514099121, 0.8989759087562561, 0.9063060879707336, 0.914563000202179, 0.914024293422699, 0.9010733366012573, 0.9217058420181274, 0.9357056021690369, 0.9354184865951538, 0.89178466796875, 0.92868971824646, 0.9109572768211365, 0.893510103225708, 0.9144385457038879, 0.9229082465171814, 0.9121864438056946, 0.8922027945518494, 0.877782940864563, 0.9027999043464661, 0.9035058617591858, 

In [27]:
import pandas as pd

# Create pandas DataFrame
eval_df = pd.DataFrame({
    'Input Prompt': results['input_text'],
    'Generated Reading': results['generated_text'],
    'Reference Reading': results['target_text']
})

# Display a few rows
eval_df.head(10)


,Input Prompt,Generated Reading,Reference Reading
0,You are a Tarot Master.\nUser Zodiac Sign: Vir...,Reading: The Eight of Swords shows you're feel...,"Virgo, the Eight of Swords suggests you feel l..."
1,You are a Tarot Master.\nUser Zodiac Sign: Aqu...,"Aquarius, the Queen and Page of Swords encoura...","Aquarius, the Queen and Page of Swords suggest..."
2,You are a Tarot Master.\nUser Zodiac Sign: Leo...,"Reading: Leo, you're the emotional glue (King ...","Reading: Leo, you're the emotional anchor and..."
3,You are a Tarot Master.\nUser Zodiac Sign: Leo...,Reading: The Page of Pentacles suggests focusi...,Reading: The Page of Pentacles suggests start...
4,You are a Tarot Master.\nUser Zodiac Sign: Vir...,"Virgo, the Eight of Wands suggests things are ...","Reading: Virgo, things are changing quickly, ..."
5,You are a Tarot Master.\nUser Zodiac Sign: Sag...,Reading: Justice suggests fairness is key to y...,Reading: Justice suggests you create a balanc...
6,You are a Tarot Master.\nUser Zodiac Sign: Sag...,"Reading: Sagittarius, the Ten of Swords sugges...",Reading: Past traumas and dramatic endings re...
7,You are a Tarot Master.\nUser Zodiac Sign: Lib...,"Libra, you're holding onto resources well, but...","Libra, the Four of Pentacles suggests you're h..."
8,You are a Tarot Master.\nUser Zodiac Sign: Pis...,"Reading: Pisces, you may feel trapped by your ...","Pisces, you're moving on from something that w..."
9,You are a Tarot Master.\nUser Zodiac Sign: Pis...,"Reading: Pisces, you're facing some challenges...","Reading: Pisces, you might face some internal..."


In [28]:
# Print question and model vs reference answer
for i in range(100):
    print(f"------------------------------")
    print(f"Prompt:\n{eval_df.iloc[i]['Input Prompt']}\n")
    print(f"Generated Reading:\n{eval_df.iloc[i]['Generated Reading']}\n")
    print(f"Reference Reading:\n{eval_df.iloc[i]['Reference Reading']}\n")
    print(f"------------------------------\n")


------------------------------
Prompt:
You are a Tarot Master.
User Zodiac Sign: Virgo
User Sentiment: negative
User's Question: Why do I feel stuck in my job?
Drawn Cards: Eight of Swords, Queen of Cups, Ten of Wands
Based on these, provide a detailed, sensitive, and inspiring Tarot Reading for each card and then summarized scentence at the end :

Generated Reading:
Reading: The Eight of Swords shows you're feeling trapped by your own thoughts. The Queen of Cups suggests you're ready and compassionate, but the Ten of Wands shows you're carrying too much responsibility. Consider if this job truly fulfills your needs.

Reference Reading:
Virgo, the Eight of Swords suggests you feel limited by self-doubt.  The Queen of Cups hints at unexpressed emotions related to your work. The Ten of Wands indicates you're carrying a heavy workload, adding to the feeling of being trapped.  Try communicating your feelings and delegating tasks.

------------------------------

---------------------------

In [29]:
from sentence_transformers import SentenceTransformer, util

# Load a sentence transformer model for embedding comparison
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Get embeddings
generated_embeddings = embedder.encode(results['generated_text'], convert_to_tensor=True)
reference_embeddings = embedder.encode(results['target_text'], convert_to_tensor=True)

# Count how many are similar enough
correct_count = 0
threshold = 0.9  # Cosine similarity threshold

for i in range(len(results['generated_text'])):
    sim = util.cos_sim(generated_embeddings[i], reference_embeddings[i])
    if sim.item() > threshold:
        correct_count += 1

precision = correct_count / len(results['generated_text'])

print('Correct (similar) count:', correct_count)
print('Total samples:', len(results['generated_text']))
print('Soft Precision Score:', round(precision, 4))


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Correct (similar) count: 253
Total samples: 1199
Soft Precision Score: 0.211


In [30]:
# Save to CSV for professional evaluation
eval_df.to_csv('/content/drive/MyDrive/NLP_project_1/tarot_model_evaluation.csv', index=False)

print("Evaluation file saved at /content/drive/MyDrive/NLP_project_1/tarot_model_evaluation.csv")


Evaluation file saved at /content/drive/MyDrive/NLP_project_1/tarot_model_evaluation.csv
